# Engine Training

In [1]:
# Importing Dependencies
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler, StandardScaler, PowerTransformer, QuantileTransformer
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from epl_pipeline import build_pipeline
from lightgbm import LGBMClassifier
from epl_datasets import load_data

import pandas as pd
import numpy as np
import warnings
import pickle
import skopt
import yaml

In [2]:
warnings.filterwarnings(action = "ignore")
pd.options.display.max_columns = None
np.random.seed(seed = 42)

Let"s load the yaml file, extract the dependent variables and assign it to a global variable.

In [3]:
# Loading the yaml file
with open(file = "../Configuration/config.yml") as yaml_file:
    config = yaml.safe_load(stream = yaml_file)

# Loading the dependent variable for Win modeling
WIN_TARGET = config.get("target").get("win_model")

# Loading the dependent variable for Loss modeling
LOSS_TARGET = config.get("target").get("loss_model")

# Loading the dependent variable for Draw modeling
DRAW_TARGET = config.get("target").get("draw_model")

Let"s call the function to load the dataset.

In [4]:
# Calling the function to load the dataset
df = load_data()

# Displaying the first five records of the dataset
df.head()

,season,match_week,match_date,month,day,weekday,referee,home_team,away_team,stadium,attendance,derby_name,club_tier_h,club_tier_a,h_position,a_position,streak_h,streak_a,goals_h,goals_a,possession_h,possession_a,shots_on_target_h,shots_on_target_a,shots_h,shots_a,touches_h,touches_a,passes_h,passes_a,tackles_h,tackles_a,clearances_h,clearances_a,corners_h,corners_a,offsides_h,offsides_a,yellow_cards_h,yellow_cards_a,red_cards_h,red_cards_a,fouls_conceded_h,fouls_conceded_a,result_h,result_a,points_h,points_a,n_epls_h,n_epls_a,n_ucls_h,n_ucls_a,total_n_matches_played_h,total_n_matches_played_a,total_max_points_h,total_max_points_a,total_points_h_cum,total_avg_acc_points_h,total_avg_points_accumulated_last_3_h,total_avg_points_accumulated_last_5_h,total_points_a_cum,total_avg_acc_points_a,total_avg_points_accumulated_last_3_a,total_avg_points_accumulated_last_5_a,total_points_dropped_h,total_avg_dropped_points_h,total_avg_points_dropped_last_3_h,total_avg_points_dropped_last_5_h,total_points_dropped_a,total_avg_dropped_points_a,total_avg_points_dropped_last_3_a,total_avg_points_dropped_last_5_a,total_goals_scored_h,total_avg_goals_scored_h,total_avg_goals_scored_last_3_h,total_avg_goals_scored_last_5_h,total_goals_scored_a,total_avg_goals_scored_a,total_avg_goals_scored_last_3_a,total_avg_goals_scored_last_5_a,total_goals_conceded_h,total_avg_goals_conceded_h,total_avg_goals_conceded_last_3_h,total_avg_goals_conceded_last_5_h,total_goals_conceded_a,total_avg_goals_conceded_a,total_avg_goals_conceded_last_3_a,total_avg_goals_conceded_last_5_a,total_avg_possession_h,total_avg_possession_a,total_avg_possession_last_3_h,total_avg_possession_last_3_a,total_avg_possession_last_5_h,total_avg_possession_last_5_a,total_shots_on_target_cum_h,total_shots_on_target_cum_a,total_avg_shots_on_target_h,total_avg_shots_on_target_a,total_avg_shots_on_target_last_3_h,total_avg_shots_on_target_last_3_a,total_avg_shots_on_target_last_5_h,total_avg_shots_on_target_last_5_a,total_shots_cum_h,total_shots_cum_a,total_avg_shots_h,total_avg_shots_a,total_avg_shots_last_3_h,total_avg_shots_last_3_a,total_avg_shots_last_5_h,total_avg_shots_last_5_a,total_avg_touches_h,total_avg_touches_a,total_avg_touches_last_3_h,total_avg_touches_last_3_a,total_avg_touches_last_5_h,total_avg_touches_last_5_a,total_avg_passes_h,total_avg_passes_a,total_avg_passes_last_3_h,total_avg_passes_last_3_a,total_avg_passes_last_5_h,total_avg_passes_last_5_a,total_avg_tackles_h,total_avg_tackles_a,total_avg_tackles_last_3_h,total_avg_tackles_last_3_a,total_avg_tackles_last_5_h,total_avg_tackles_last_5_a,total_avg_clearances_h,total_avg_clearances_a,total_avg_clearances_last_3_h,total_avg_clearances_last_3_a,total_avg_clearances_last_5_h,total_avg_clearances_last_5_a,total_avg_corners_h,total_avg_corners_a,total_avg_corners_last_3_h,total_avg_corners_last_3_a,total_avg_corners_last_5_h,total_avg_corners_last_5_a,total_avg_offsides_h,total_avg_offsides_a,total_avg_offsides_last_3_h,total_avg_offsides_last_3_a,total_avg_offsides_last_5_h,total_avg_offsides_last_5_a,total_avg_yellow_cards_h,total_avg_yellow_cards_a,total_avg_yellow_cards_last_3_h,total_avg_yellow_cards_last_3_a,total_avg_yellow_cards_last_5_h,total_avg_yellow_cards_last_5_a,total_avg_fouls_conceded_h,total_avg_fouls_conceded_a,total_avg_fouls_conceded_last_3_h,total_avg_fouls_conceded_last_3_a,total_avg_fouls_conceded_last_5_h,total_avg_fouls_conceded_last_5_a,total_s2g_cum_ratio_h,total_s2g_ratio_last_3_h,total_s2g_ratio_last_5_h,total_s2g_cum_ratio_a,total_s2g_ratio_last_3_a,total_s2g_ratio_last_5_a,total_s2s_cum_ratio_h,total_s2s_ratio_last_3_h,total_s2s_ratio_last_5_h,total_s2s_cum_ratio_a,total_s2s_ratio_last_3_a,total_s2s_ratio_last_5_a,n_matches_played_h,n_matches_played_a,max_points_h,max_points_a,points_h_cum,avg_acc_points_h,avg_points_accumulated_last_3_h,avg_points_accumulated_last_5_h,points_a_cum,avg_acc_points_a,avg_points_accumulated_last_3_a,avg_points_accumulated_last_5_a,points_dropped_h,avg_dropped_p

Let's create train features and labels for each of the models.

In [5]:
# Creating train features for each of the models
X_train_win = df.drop(columns = WIN_TARGET)
X_train_loss = df.drop(columns = LOSS_TARGET)
X_train_draw = df.drop(columns = DRAW_TARGET)

# Creating train labels for each of the models
y_train_win = df[WIN_TARGET]
y_train_loss = df[LOSS_TARGET]
y_train_draw = df[DRAW_TARGET]

***
### Win Modeling

Let's call the function to build a classifier pipeline using **Logistic Regression** algorithm.

In [6]:
# Defining an operating level seed
np.random.seed(seed = 42)

# Creating a list of feature scaler instances
feature_scalers = [RobustScaler(), MinMaxScaler(), MaxAbsScaler(), StandardScaler(), PowerTransformer(), QuantileTransformer()]

# Creating a dictionary of hyperparameters
search_spaces = {}
search_spaces["feature_transformer__numeric_pipeline__feature_scaler"] = feature_scalers
search_spaces["feature_selector__percentile"] = skopt.space.Categorical(categories = list(range(10, 91, 10)))
search_spaces["classifier__C"] = skopt.space.Real(low = 1e-6, high = 100, prior = "log-uniform")
search_spaces["classifier__tol"] = skopt.space.Real(low = 1e-6, high = 100, prior = "log-uniform")

# Building a classifier
lr_model_win = build_pipeline(classifier = LogisticRegression(solver = "liblinear", random_state = 42), 
                              apply_feature_scaling = True,
                              feature_scaler_type = "robust",
                              winsorize_outliers = True,
                              apply_feature_selection = True,
                              feature_selection_type = "mutual_info",
                              apply_bayesian_optimization = True,
                              hyperparameters = search_spaces,
                              n_iterations = 100,
                              train_features = X_train_win, 
                              train_labels = y_train_win,
                              verbosity = 1)

# Displaying the architecture of the classifier
lr_model_win

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

Pipeline(steps=[('initial_preprocessor', InitialPreprocessor()),
                ('feature_transformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('nominal_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['derby_name', 'home_team',
                                                   'away_team']),
                                                 ('ordinal_pipeline',
                                                  P...
                                                   'total_avg_goals_scored_last_3_h',
                                                   'total_avg_goals_scored_last_3_a',
                                                   'total_avg_goals_scored_last_5_h',
                                                   'total_avg_goals_scored_last_5_a',
                                                   'total_goals_conceded_h',
                                                   'total_goals_conceded_a', ...])])),
                ('feature_selector', SelectPercentile(percentile=50)),
                ('classifier',
                 LogisticRegression(C=0.007605329373330636, random_state=42,
                                    solver='liblinear',
                                    tol=1.6614871294767138e-06))])

Let's call the function to build a classifier pipeline using **Multi Layer Perceptron** algorithm.

In [7]:
# Defining an operating level seed
np.random.seed(seed = 42)

# Creating a dictionary of hyperparameters
search_spaces = {}
search_spaces["feature_transformer__numeric_pipeline__feature_scaler"] = feature_scalers
search_spaces["feature_selector__percentile"] = skopt.space.Categorical(categories = list(range(10, 91, 10)))
search_spaces["classifier__alpha"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__learning_rate"] = skopt.space.Categorical(categories = ["constant", "invscaling", "adaptive"])
search_spaces["classifier__learning_rate_init"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__tol"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")

# Building a classifier
mlp_model_win = build_pipeline(classifier = MLPClassifier(learning_rate = "adaptive", 
                                                          max_iter = 1000, 
                                                          shuffle = False, 
                                                          early_stopping = True,
                                                          n_iter_no_change = 3),  
                               apply_feature_scaling = True,
                               feature_scaler_type = "standard",
                               apply_feature_selection = True,
                               feature_selection_type = "mutual_info",
                               apply_bayesian_optimization = True,
                               hyperparameters = search_spaces,
                               n_iterations = 100,
                               train_features = X_train_win, 
                               train_labels = y_train_win, 
                               verbosity = 1)

# Displaying the architecture of the classifier
mlp_model_win

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

Pipeline(steps=[('initial_preprocessor', InitialPreprocessor()),
                ('feature_transformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('nominal_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['derby_name', 'home_team',
                                                   'away_team']),
                                                 ('ordinal_pipeline',
                                                  P...
                                                   'total_avg_goals_scored_last_5_h',
                                                   'total_avg_goals_scored_last_5_a',
                                                   'total_goals_conceded_h',
                                                   'total_goals_conceded_a', ...])])),
                ('feature_selector', SelectPercentile(percentile=50)),
                ('classifier',
                 MLPClassifier(alpha=0.1, early_stopping=True,
                               learning_rate='adaptive',
                               learning_rate_init=0.00904369976619128,
                               max_iter=1000, n_iter_no_change=3, shuffle=False,
                               tol=0.1))])

Let's call the function to build a classifier pipeline using **Light Gradient Boosted Machine (LightGBM)** algorithm.

In [8]:
# Defining an operating level seed
np.random.seed(seed = 42)

# Creating a dictionary of hyperparameters
search_spaces = {}
search_spaces["feature_selector__threshold"] = skopt.space.Categorical(categories = [None, "median"])
search_spaces["classifier__max_depth"] = skopt.space.Integer(low = 1, high = 6)
search_spaces["classifier__learning_rate"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__n_estimators"] = skopt.space.Integer(low = 100, high = 300)
search_spaces["classifier__reg_alpha"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__reg_lambda"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")

# Building a classifier
lgbm_model_win = build_pipeline(classifier = LGBMClassifier(objective = "binary", random_state = 42, n_jobs = -1),  
                                apply_feature_selection = True,
                                feature_selection_type = "meta",
                                apply_bayesian_optimization = True,
                                hyperparameters = search_spaces,
                                n_iterations = 100,
                                train_features = X_train_win, 
                                train_labels = y_train_win, 
                                verbosity = 1)
 
# Displaying the architecture of the classifier
lgbm_model_win

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

Pipeline(steps=[('initial_preprocessor', InitialPreprocessor()),
                ('feature_transformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('nominal_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['derby_name', 'home_team',
                                                   'away_team']),
                                                 ('ordinal_pipeline',
                                                  P...
                                                   'total_avg_goals_scored_last_5_h',
                                                   'total_avg_goals_scored_last_5_a',
                                                   'total_goals_conceded_h',
                                                   'total_goals_conceded_a', ...])])),
                ('feature_selector',
                 SelectFromModel(estimator=LGBMClassifier(objective='binary',
                                                          random_state=42),
                                 threshold='median')),
                ('classifier',
                 LGBMClassifier(max_depth=1, n_estimators=136,
                                objective='binary', random_state=42,
                                reg_alpha=0.1, reg_lambda=1e-08))])

Let's create a **stacked** model containing previously trained three models.

In [9]:
# Creating a list of estimators
win_models = [('Logistic Regression', lr_model_win), 
              ('Light Gradient Boosted Machine', lgbm_model_win), 
              ('Multi Layer Perceptron', mlp_model_win)]

# Creating an estimator of stacked classifier
win_model = StackingClassifier(estimators = win_models, n_jobs = -1)

# Fitting the train features and labels
win_model.fit(X = X_train_win, y = y_train_win)

StackingClassifier(estimators=[('Logistic Regression',
                                Pipeline(steps=[('initial_preprocessor',
                                                 InitialPreprocessor()),
                                                ('feature_transformer',
                                                 ColumnTransformer(n_jobs=-1,
                                                                   remainder='passthrough',
                                                                   transformers=[('nominal_pipeline',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('ohe',
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['derby_...
                                                                                   'total_avg_goals_scored_last_5_h',
                                                                                   'total_avg_goals_scored_last_5_a',
                                                                                   'total_goals_conceded_h',
                                                                                   'total_goals_conceded_a', ...])])),
                                                ('feature_selector',
                                                 SelectPercentile(percentile=50)),
                                                ('classifier',
                                                 MLPClassifier(alpha=0.1,
                                                               early_stopping=True,
                                                               learning_rate='adaptive',
                                                               learning_rate_init=0.00904369976619128,
                                                               max_iter=1000,
                                                               n_iter_no_change=3,
                                                               shuffle=False,
                                                               tol=0.1))]))],
                   n_jobs=-1)

***
### Loss Modeling

Let's call the function to build a classifier pipeline using **Logistic Regression** algorithm.

In [10]:
# Defining an operating level seed
np.random.seed(seed = 42)

# Creating a dictionary of hyperparameters
search_spaces = {}
search_spaces["feature_transformer__numeric_pipeline__feature_scaler"] = feature_scalers
search_spaces["feature_selector__percentile"] = skopt.space.Categorical(categories = list(range(10, 91, 10)))
search_spaces["classifier__C"] = skopt.space.Real(low = 1e-6, high = 100, prior = "log-uniform")
search_spaces["classifier__tol"] = skopt.space.Real(low = 1e-6, high = 100, prior = "log-uniform")

# Building a classifier
lr_model_loss = build_pipeline(classifier = LogisticRegression(solver = "liblinear", random_state = 42), 
                               apply_feature_scaling = True,
                               feature_scaler_type = "robust",
                               winsorize_outliers = True,
                               apply_feature_selection = True,
                               feature_selection_type = "mutual_info",
                               apply_bayesian_optimization = True,
                               hyperparameters = search_spaces,
                               n_iterations = 100,
                               train_features = X_train_loss, 
                               train_labels = y_train_loss,
                               verbosity = 1)

# Displaying the architecture of the classifier
lr_model_loss

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

Pipeline(steps=[('initial_preprocessor', InitialPreprocessor()),
                ('feature_transformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('nominal_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['derby_name', 'home_team',
                                                   'away_team']),
                                                 ('ordinal_pipeline',
                                                  P...
                                                   'total_avg_goals_scored_last_3_h',
                                                   'total_avg_goals_scored_last_3_a',
                                                   'total_avg_goals_scored_last_5_h',
                                                   'total_avg_goals_scored_last_5_a',
                                                   'total_goals_conceded_h',
                                                   'total_goals_conceded_a', ...])])),
                ('feature_selector', SelectPercentile(percentile=40)),
                ('classifier',
                 LogisticRegression(C=0.010222809578331165, random_state=42,
                                    solver='liblinear',
                                    tol=0.0006255557372217619))])

Let's call the function to build a classifier pipeline using **Multi Layer Perceptron** algorithm.

In [11]:
# Defining an operating level seed
np.random.seed(seed = 42)

# Creating a dictionary of hyperparameters
search_spaces = {}
search_spaces["feature_transformer__numeric_pipeline__feature_scaler"] = feature_scalers
search_spaces["feature_selector__percentile"] = skopt.space.Categorical(categories = list(range(10, 91, 10)))
search_spaces["classifier__alpha"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__learning_rate"] = skopt.space.Categorical(categories = ["constant", "invscaling", "adaptive"])
search_spaces["classifier__learning_rate_init"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__tol"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")

# Building a classifier
mlp_model_loss = build_pipeline(classifier = MLPClassifier(learning_rate = "adaptive", 
                                                           max_iter = 1000, 
                                                           shuffle = False, 
                                                           early_stopping = True,
                                                           n_iter_no_change = 3),  
                                apply_feature_scaling = True,
                                feature_scaler_type = "standard",
                                apply_feature_selection = True,
                                feature_selection_type = "mutual_info",
                                apply_bayesian_optimization = True,
                                hyperparameters = search_spaces,
                                n_iterations = 100,
                                train_features = X_train_loss, 
                                train_labels = y_train_loss, 
                                verbosity = 1)

# Displaying the architecture of the classifier
mlp_model_loss

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

Pipeline(steps=[('initial_preprocessor', InitialPreprocessor()),
                ('feature_transformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('nominal_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['derby_name', 'home_team',
                                                   'away_team']),
                                                 ('ordinal_pipeline',
                                                  P...
                                                   'total_avg_goals_scored_last_5_h',
                                                   'total_avg_goals_scored_last_5_a',
                                                   'total_goals_conceded_h',
                                                   'total_goals_conceded_a', ...])])),
                ('feature_selector', SelectPercentile(percentile=50)),
                ('classifier',
                 MLPClassifier(alpha=0.1, early_stopping=True,
                               learning_rate='invscaling',
                               learning_rate_init=0.0074145848140236715,
                               max_iter=1000, n_iter_no_change=3, shuffle=False,
                               tol=1e-08))])

Let's call the function to build a classifier pipeline using **Light Gradient Boosted Machine (LightGBM)** algorithm.

In [12]:
# Defining an operating level seed
np.random.seed(seed = 42)

# Creating a dictionary of hyperparameters
search_spaces = {}
search_spaces["feature_selector__threshold"] = skopt.space.Categorical(categories = [None, "median"])
search_spaces["classifier__max_depth"] = skopt.space.Integer(low = 1, high = 6)
search_spaces["classifier__learning_rate"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__n_estimators"] = skopt.space.Integer(low = 100, high = 300)
search_spaces["classifier__reg_alpha"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__reg_lambda"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")

# Building a classifier
lgbm_model_loss = build_pipeline(classifier = LGBMClassifier(objective = "binary", random_state = 42, n_jobs = -1),  
                                 apply_feature_selection = True,
                                 feature_selection_type = "meta",
                                 apply_bayesian_optimization = True,
                                 hyperparameters = search_spaces,
                                 n_iterations = 100,
                                 train_features = X_train_loss, 
                                 train_labels = y_train_loss, 
                                 verbosity = 1)
 
# Displaying the architecture of the classifier
lgbm_model_loss

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

Pipeline(steps=[('initial_preprocessor', InitialPreprocessor()),
                ('feature_transformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('nominal_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['derby_name', 'home_team',
                                                   'away_team']),
                                                 ('ordinal_pipeline',
                                                  P...
                                                   'total_avg_goals_scored_last_5_a',
                                                   'total_goals_conceded_h',
                                                   'total_goals_conceded_a', ...])])),
                ('feature_selector',
                 SelectFromModel(estimator=LGBMClassifier(objective='binary',
                                                          random_state=42),
                                 threshold='median')),
                ('classifier',
                 LGBMClassifier(learning_rate=0.01596103700861989, max_depth=2,
                                n_estimators=300, objective='binary',
                                random_state=42, reg_alpha=1e-08,
                                reg_lambda=0.1))])

Let's build **Voting** model based on majority vote using previously trained models.

In [13]:
# Creating a list of estimators
loss_models = [('Logistic Regression', lr_model_loss), 
               ('Light Gradient Boosted Machine', lgbm_model_loss), 
               ('Multi Layer Perceptron', mlp_model_loss)]

# Creating an estimator of voting classifier
loss_model = VotingClassifier(estimators = loss_models, voting = 'soft', n_jobs = -1)

# Fitting the train features and labels
loss_model.fit(X = X_train_loss, y = y_train_loss)

VotingClassifier(estimators=[('Logistic Regression',
                              Pipeline(steps=[('initial_preprocessor',
                                               InitialPreprocessor()),
                                              ('feature_transformer',
                                               ColumnTransformer(n_jobs=-1,
                                                                 remainder='passthrough',
                                                                 transformers=[('nominal_pipeline',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='most_frequent')),
                                                                                                ('ohe',
                                                                                                 OneHotEncoder(handle_unknown='ignore'))]),
                                                                                ['derby_na...
                                                                                 'total_avg_goals_scored_last_5_a',
                                                                                 'total_goals_conceded_h',
                                                                                 'total_goals_conceded_a', ...])])),
                                              ('feature_selector',
                                               SelectPercentile(percentile=50)),
                                              ('classifier',
                                               MLPClassifier(alpha=0.1,
                                                             early_stopping=True,
                                                             learning_rate='invscaling',
                                                             learning_rate_init=0.0074145848140236715,
                                                             max_iter=1000,
                                                             n_iter_no_change=3,
                                                             shuffle=False,
                                                             tol=1e-08))]))],
                 n_jobs=-1, voting='soft')

***
### Draw Modeling

Let's call the function to build a classifier pipeline using **Multi Layer Perceptron** algorithm.

In [14]:
# Defining an operating level seed
np.random.seed(seed = 42)

# Creating a dictionary of hyperparameters
search_spaces = {}
search_spaces["feature_transformer__numeric_pipeline__feature_scaler"] = feature_scalers
search_spaces["feature_selector__percentile"] = skopt.space.Categorical(categories = list(range(10, 91, 10)))
search_spaces["classifier__alpha"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__learning_rate"] = skopt.space.Categorical(categories = ["constant", "invscaling", "adaptive"])
search_spaces["classifier__learning_rate_init"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")
search_spaces["classifier__tol"] = skopt.space.Real(low = 1e-8, high = 0.1, prior = "log-uniform")

# Building a classifier
draw_model = build_pipeline(classifier = MLPClassifier(learning_rate = "adaptive", 
                                                       max_iter = 1000, 
                                                       shuffle = False, 
                                                       early_stopping = True,
                                                       n_iter_no_change = 3),  
                            apply_feature_scaling = True,
                            feature_scaler_type = "standard",
                            apply_feature_selection = True,
                            feature_selection_type = "mutual_info",
                            apply_bayesian_optimization = True,
                            hyperparameters = search_spaces,
                            n_iterations = 100,
                            train_features = X_train_draw, 
                            train_labels = y_train_draw, 
                            verbosity = 1)

# Displaying the architecture of the classifier
draw_model

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

Pipeline(steps=[('initial_preprocessor', InitialPreprocessor()),
                ('feature_transformer',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('nominal_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['derby_name', 'home_team',
                                                   'away_team']),
                                                 ('ordinal_pipeline',
                                                  P...
                                                   'total_avg_goals_scored_last_5_h',
                                                   'total_avg_goals_scored_last_5_a',
                                                   'total_goals_conceded_h',
                                                   'total_goals_conceded_a', ...])])),
                ('feature_selector', SelectPercentile(percentile=90)),
                ('classifier',
                 MLPClassifier(alpha=0.1, early_stopping=True,
                               learning_rate='invscaling',
                               learning_rate_init=0.005342343782828189,
                               max_iter=1000, n_iter_no_change=3, shuffle=False,
                               tol=0.1))])

Let's save the classifier pipelines as pickle objects.

In [15]:
# Saving the classifier pipeline for the Win model as pickle object
with open(file = '../Models/win_model.pickle', mode = 'wb') as pickled_model:
    pickle.dump(obj = win_model, file = pickled_model)
    
# Saving the classifier pipeline for the Loss model as pickle object
with open(file = '../Models/loss_model.pickle', mode = 'wb') as pickled_model:
    pickle.dump(obj = loss_model, file = pickled_model)
    
# Saving the classifier pipeline for the Draw model as pickle object
with open(file = '../Models/draw_model.pickle', mode = 'wb') as pickled_model:
    pickle.dump(obj = draw_model, file = pickled_model)